# ISYE 6501 Homework #8

## Jeremy Wong | kwong301@gatech.edu

# Question 11.1

Using the crime data set `uscrime.txt` from Questions 8.2, 9.1, and 10.1, build a regression model using:
1. Stepwise regression
2. Lasso
3. Elastic net

For Parts 2 and 3, remember to scale the data first – otherwise, the regression coefficients will be on different scales and the constraint won’t have the desired effect. 

For Parts 2 and 3, use the `glmnet` function in R. Notes on R:
- For the elastic net model, what we called λ in the videos, glmnet calls “alpha”; you can get a range of results by varying alpha from 1 (lasso) to 0 (ridge regression) and, of course, other values of alpha in between.
- In a function call like glmnet(x,y,family=”mgaussian”,alpha=1) the predictors x need to be in R’s matrix format, rather than data frame format.  You can convert a data frame to a matrix using as.matrix – for example, x <- as.matrix(data[,1:n-1])
- Rather than specifying a value of T, glmnet returns models for a variety of values of T. 


# Answers to 11.1

## Summary
- Discussed the methodology, inlucding a review on literature criticisms of stepwise regression in section 1.1.
- R implementation discussed in section 1.2, with a introduction of `caret` for future usage.
- Reported final model in section 1.3.
- Discussed the implemention of glmnet in section 2.1.
- Reported final model in section 2.2.

## Conclusion
- Stepwise model suggested using `M`, `Ed`, `Po1`, `M.F`, `U1`, `U2`, `Ineq`, `Prob` as regressors. 
- In the final Lasso model, `M`, `So`, `Ed`, `Po1`, `LF`, `M.F`, `NW`, `U1`, `U2`, `Ineq`, `Prob` are selected, while in ridge regression all regressors are selected with shrinked coefficients.
- We see that the Lasso model is selected among the class of elastic net models - suggesting that dropping variables in our case here is preferred.
- In general, we do not suggest using stepwise regression for formal analysis, because of its issues with data dredging and spurious inference. 
- Using LOOCV errors alone, we will prefer the Lasso model if we are to choose from CV errors. 

| Model | Mean CV Error | No. of variables |
|:-----:|:-------------:|:----------------:|
| Stepwise | 71499      | 8
| Lasso | 65813         | 11               |
| Ridge | 67889         | 15               |
| Elnet | 65813         | 11               |

# Details to Answers

# 1.1 Stepwise Regression

Coming from an Econometrics background, I'm not very fond of automated stepwise regression approaches that are largely greedy in nature, offer false confidence, and doesn't give insights to model interpretation. [This post](https://stats.stackexchange.com/questions/20836/algorithms-for-automatic-model-selection/20856#20856) and [this paper](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-018-0143-6) offer elaborated explanations on why stepwise regression methods can be bad. Essentially, the $p$-values under the t and F tests do not have the claimed distributions in the subsamples for the intermediate/final steps (section 4.3 in [Harrel's notes](https://hbiostat.org/doc/rms.pdf)). Even if selection criterion may not involve p-values/metrics with underlying distribution assumption, the $p$ values and confidence tend to be [over-confident](https://onlinelibrary.wiley.com/doi/epdf/10.1002/sim.4780080702), i.e. too small/narrow - because they are chosen according to the performance in tests, creating the illusion that "significant" relationships between the variables have been found (in other words, $\mathbb{E}[\beta|p_\beta < 0.05] \neq \mathbb{E}[\beta]$). Also check [this Stata memo](https://www.stata.com/support/faqs/statistics/stepwise-regression-problems/) for review of criticisms to this approach in the literature. 

These shortcomings typically [arise from interactions between correlated regressors](https://onlinelibrary.wiley.com/doi/epdf/10.1002/sim.4780080702). [Heinze, Wallisch and Dunkler (2018)](https://onlinelibrary.wiley.com/doi/full/10.1002/bimj.201700067) provide a nice summary on the mechanism. Check [here](https://www3.nd.edu/~rwilliam/stats1/x91.pdf) for a review on how correlation between regressors affects their standard errors.

<img src="https://onlinelibrary.wiley.com/cms/asset/a7dc40a8-60b0-48d9-8f35-32c3256d5be0/bimj1842-fig-0002-m.jpg" width="50%">

However, if we are just interested in using a simplified model to make predictions, it would still worth to explore these methods. We introduce three heuristic methods for selecting features in a linear regression model and we will apply the third method, Stepwise Regression which is a generalization of the first two, on the crime data.

### Forward variable selection

Add factors in the regression until the model performance is satisfactory or we reach the upper bound for the number of variables preset by the user and then trim the model _greedily_ depending on the p-value of each individual variable, as shown in the procedure diagram below.

<img src="https://github.com/kpjwong/ISYE-6501/blob/main/images/forward_selection.png?raw=true" width="70%">

### Backward variable elimination

From a full model with all variables, remove factors until the number of regressors is within a range preset by the user and then stop dropping when we observe a statistically significant variable, as shown below.

<img src="https://github.com/kpjwong/ISYE-6501/blob/main/images/backward_elimination.png?raw=true" width="70%">

### Stepwise Regression

Note that "stepwise regression" is a generalization  to variable selection combining the two _greedy_ approaches above. Essentially, it follows the foward selection routine - until we have enough valid variables or the model performance is good enough, we have a combined operation of adding the candidate regressor and eliminate statistically insignificant variables. _Note: I believe there is an error in the "Yes" and "No" for the "Is it good enough?" decision node._

<img src="https://github.com/kpjwong/ISYE-6501/blob/main/images/stepwise_regression.png?raw=true" width="70%">

### Model Performance Criteria

We need some criteria for model performance to make decisions in the decision nodes. Here is a summary with some common criteria:

- p-value: This is also the criterion suggested by the slides above. Currently, no R packages support this version. A function written by Paul Rubin supports stepwise regression with this [blog post](https://orinanobworld.blogspot.com/2011/02/stepwise-regression-in-r.html).
- AIC: In each step, the AIC's before and after are compared. Variables are added/eliminated based on reduction in AIC, or a cp (complexity parameter) derived from AIC. In R, this is supported by the built-in `step` function and also the `StepAIC` in `MASS` pacakge. These functions has CV wrapper support by `caret` under the alias of `lmStepAIC` and `glmStepAIC`. 
- Mallow's CP and (gains in) R Squared: both are supported by `regsubsets` function under `leaps` and a `caret` CV wrapper (with alis `leapSeq`). [Mallow's CP](https://support.minitab.com/en-us/minitab/18/help-and-how-to/modeling-statistics/regression/supporting-topics/goodness-of-fit-statistics/what-is-mallows-cp/) is a metric similar to AIC which accounts for both goodness-of-fit and model compexity.

I am a bit skeptical about using individual p-values for each step greedily. Correlation among variables could give spurious (non-)confidence. E.g. variables can be jointly significant but insignificant individually. AIC is likelihood-based, we've seen in homework 6 that the QQ plots for the residuals look close to normal distribution when we have around 6 or more variables. This justifies the use of AIC. An alternative way would be to use the model-agnostic Mallow's CP for model selection. 

# 1.2 Stepwise Regression Implementation

We use the `lmStepAIC` model in `caret` for CV. The base `StepAIC` function:

```R
stepAIC(object, scope, scale = 0,
        direction = c("both", "backward", "forward"),
        trace = 1, keep = NULL, steps = 1000, use.start = FALSE,
        k = 2, …)
```

- `object` needs to be a model object, specifying the base model to be considered in the first step. An example is `lm(y ~ ., data)`, a linear regression with the response regressed on all variables.
- `scope` is a list with names upper and lower, which are model objects specifying the scope of search. The upper model will include all candidate variables, while the lower model will include all variables that must be included.
- `direction` specifies the mode of stepwise search, can be one of "both", "backward", or "forward", with a default of "both". If the scope argument is missing the default for direction is "backward".

Essentially, the `StepAIC` function internalizes the AIC selection process up to the above input parameters. With the `caret` wrapper, we will just need to specify the CV control/train setup. We'll use a LOOCV and a 10-fold CV repeated 5 times. The `train` function syntax can either be in terms of X and y, or a formula and data.

```R
## S3 method for class 'default':
train(x, y, 
      method = "rf",  
      ..., 
      weights = NULL,
      metric = ifelse(is.factor(y), "Accuracy", "RMSE"),   
      maximize = ifelse(metric == "RMSE", FALSE, TRUE),
      trControl = trainControl(), 
      tuneGrid = NULL, 
      tuneLength = 3)

## S3 method for class 'formula':
train(form, data, ..., weights, subset, na.action, contrasts = NULL)
```

Here we specify the `method` as `lmStepAIC`, the CV scheme (10 fold, or LOO) is specified in `trControl`. We will use RMSE for the model selection criterion (in CV). Other parameters to be passed to the base method should be included in `...`. For other usage of the package refer to the [offical documentation](https://topepo.github.io/caret/model-training-and-tuning.html).

# 1.3 Stepwise Regression Model Selection

We see that both the 10-fold CV and the LOOCV suggest using `M`, `Ed`, `Po1`, `M.F`, `U1`, `U2`, `Ineq`, `Prob` as regressors. We've seen that `U1` and `U2` are highly correlated.

In [46]:
library(data.table)
library(caret)
library(magrittr)

crime_dta <- fread('./hw8/uscrime.txt')

K_fold_ctrl <- trainControl(method="repeatedcv", number=10, repeats=5)
LOO_ctrl <- trainControl(method = "LOOCV")

set.seed(1)
K_fold_stepreg <- train(Crime ~., data = crime_dta ,
                        method = "lmStepAIC", 
                        trControl = K_fold_ctrl, trace=FALSE
                        )
K_fold_stepreg$finalModel


Call:
lm(formula = .outcome ~ M + Ed + Po1 + M.F + U1 + U2 + Ineq + 
    Prob, data = dat)

Coefficients:
(Intercept)            M           Ed          Po1          M.F           U1  
   -6426.10        93.32       180.12       102.65        22.34     -6086.63  
         U2         Ineq         Prob  
     187.35        61.33     -3796.03  


In [94]:
LOO_stepreg <- train(Crime ~., data = crime_dta ,
                     method = "lmStepAIC", 
                     trControl = LOO_ctrl, trace=FALSE
                        )
LOO_stepreg$finalModel


Call:
lm(formula = .outcome ~ M + Ed + Po1 + M.F + U1 + U2 + Ineq + 
    Prob, data = dat)

Coefficients:
(Intercept)            M           Ed          Po1          M.F           U1  
   -6426.10        93.32       180.12       102.65        22.34     -6086.63  
         U2         Ineq         Prob  
     187.35        61.33     -3796.03  


In [169]:
LOO_stepreg %>% names

[1] "method"       "modelInfo"    "modelType"    "results"      "pred"        
 [6] "bestTune"     "call"         "dots"         "metric"       "control"     
[11] "finalModel"   "preProcess"   "trainingData" "resample"     "resampledCM" 
[16] "perfNames"    "maximize"     "yLimits"      "times"        "levels"      
[21] "terms"        "coefnames"    "xlevels"

In [175]:
# CV err
(LOO_stepreg$results$RMSE)^2

[1] 71498.88

### Final Model

Train on the entire data.

In [53]:
 lm(Crime ~ M + Ed + Po1 + M.F + U1 + U2 + Ineq + Prob, crime_dta) %>% summary


Call:
lm(formula = Crime ~ M + Ed + Po1 + M.F + U1 + U2 + Ineq + Prob, 
    data = crime_dta)

Residuals:
    Min      1Q  Median      3Q     Max 
-444.70 -111.07    3.03  122.15  483.30 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -6426.10    1194.61  -5.379 4.04e-06 ***
M              93.32      33.50   2.786  0.00828 ** 
Ed            180.12      52.75   3.414  0.00153 ** 
Po1           102.65      15.52   6.613 8.26e-08 ***
M.F            22.34      13.60   1.642  0.10874    
U1          -6086.63    3339.27  -1.823  0.07622 .  
U2            187.35      72.48   2.585  0.01371 *  
Ineq           61.33      13.96   4.394 8.63e-05 ***
Prob        -3796.03    1490.65  -2.547  0.01505 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 195.5 on 38 degrees of freedom
Multiple R-squared:  0.7888,	Adjusted R-squared:  0.7444 
F-statistic: 17.74 on 8 and 38 DF,  p-value: 1.159e-10


In [55]:
 lm(Crime ~ M + Ed + Po1 + M.F + U2 + Ineq + Prob, crime_dta) %>% summary


Call:
lm(formula = Crime ~ M + Ed + Po1 + M.F + U2 + Ineq + Prob, data = crime_dta)

Residuals:
    Min      1Q  Median      3Q     Max 
-433.83 -121.23    2.28  125.26  551.58 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -5666.91    1152.50  -4.917 1.63e-05 ***
M              97.87      34.38   2.846  0.00701 ** 
Ed            170.22      54.01   3.151  0.00312 ** 
Po1           116.58      13.91   8.382 2.95e-10 ***
M.F            10.84      12.40   0.874  0.38750    
U2             79.01      42.71   1.850  0.07190 .  
Ineq           65.69      14.16   4.640 3.87e-05 ***
Prob        -3858.20    1533.99  -2.515  0.01613 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 201.3 on 39 degrees of freedom
Multiple R-squared:  0.7704,	Adjusted R-squared:  0.7291 
F-statistic: 18.69 on 7 and 39 DF,  p-value: 1.173e-10


# 2.1 Lasso/Elastic Net Implementation

Lasso and elastic net implementation belongs to the class of regularized linear regression, which minimizes the following regularized loss function:

\begin{align*}
L(\beta; \alpha, \lambda) &= \sum_{i=1}^n (y_i - \beta x_i)^2 + \lambda \bigg[ \alpha \Vert \beta \Vert_1 + \frac{1-\alpha}{2} \Vert \beta \Vert^2_2 \bigg]
\end{align*}

- $\lambda \geq 0$ is the complexity parameter, $\alpha \in [0,1]$ determines the model specification.
- If $\alpha = 0$, it is a ridge regression.
- If $\alpha = 1$, it is a lasso regression.
- If $\alpha \in (0,1)$, it is an elastic net.

We'll use the R package `glmnet` to train the model. The R syntax is:

```R
glmnet(
    x,
    y,
    family = c("gaussian", "binomial", "poisson", "multinomial", "cox", "mgaussian"),
    alpha = 1,
    nlambda = 100,
    lambda.min.ratio = ifelse(nobs < nvars, 0.01, 1e-04),
    standardize = TRUE,
    thresh = 1e-07,
    dfmax = nvars + 1,
    pmax = min(dfmax * 2 + 20, nvars),
    ...
)
```

Notable input parameters:

- `x`, `y` are the independent and response variables. Note that they need to be in matrix forms.
- `family` specifies the distribution of the error term, which 
- `alpha` is the same as the parameter $\alpha$ in the equation above. Zero imples ridge regression and one implies lasso.
- `nlambda` is the number of potential complexity parameter $\lambda$ to be evaluated. Note that `glmnet` internalize the selection of $\lambda$. 
- `lambda.min.ratio` is the ratio of minimum lambda relative to the maximum lambda to be considered. Note that the maximum lambda is the smallest value for which all the coefficients in $\beta$ are zero.
- `standardize` is a logical flag for x variable standardization, prior to fitting the model sequence. The coefficients are always returned on the original scale, so we don't have to do this beforehand.
- `thresh` is the convergence criterion for the algorithm.
- `dfmax` is the maximum limit on the number of variables in the model. Useful for very large nvars, if a partial path is desired.
- `pmax` is the the maximum number of variables ever allowed to be nonzero.

Notable outputs and methods:

- `.$dev.ratio`: The fraction of deviance explained / R squared in the case of elastic net / Lasso or ridge.
- `.$nulldev`: The per-observation deviance under null (intercept only model), this can be used to derive the model RMSE.
- `print()`: Print the deviance ratio for each step.
- `predict()`: Make predictions at given values of X.
- `plot()`: Plot the coefficients across lambdas.
- `cv.glmnet()`: A built-in CV routine to select glmnet model with the best lambda. Refer [here](https://web.stanford.edu/~hastie/glmnet/glmnet_alpha.html) for a demo.

`caret` does provide wrapper support for glmnet. It allows us to use CV to tune lambda and alpha simultaneously through a grid search (grid points specified by the `tuneGrid` parameter). It is however noted that the [lambda output is probably not optimized](https://stats.stackexchange.com/questions/327239/r-coefficients-from-glmnet-and-caret-are-different-for-the-same-lambda). So we'll iterate alpha over a loop of `cv.glmnet` calls. We'll use LOOCV.

# 2.2 Results

Here's the pure lasso, ridge and elastic net model selection results using LOOCV. Note that the Elastic Net had the lowest mean CV error at alpha=1, which is the lasso case.

| Model | Mean CV Error | No. of variables |
|:-----:|:-------------:|:----------------:|
| Lasso | 65813         | 11               |
| Ridge | 67889         | 15               |
| Elnet | 65813         | 11               |

In [156]:
lasso_mod <- cv.glmnet(x=crime_dta[, -"Crime"] %>% as.matrix,
                       y=crime_dta[, "Crime"] %>% as.matrix,
                       nfolds=nrow(crime_dta),
                       alpha=1, family="gaussian")

lasso_mod

Warning message:
"Option grouped=FALSE enforced in cv.glmnet, since < 3 observations per fold"



Call:  cv.glmnet(x = crime_dta[, -"Crime"] %>% as.matrix, y = crime_dta[,      "Crime"] %>% as.matrix, nfolds = nrow(crime_dta), alpha = 1,      family = "gaussian") 

Measure: Mean-Squared Error 

    Lambda Index Measure    SE Nonzero
min  13.40    33   65813 13349      11
1se  37.29    22   76484 15389       5

In [158]:
ridge_mod <- cv.glmnet(x=crime_dta[, -"Crime"] %>% as.matrix,
                       y=crime_dta[, "Crime"] %>% as.matrix,
                       nfolds=nrow(crime_dta),
                       alpha=0, family="gaussian")

ridge_mod

Warning message:
"Option grouped=FALSE enforced in cv.glmnet, since < 3 observations per fold"



Call:  cv.glmnet(x = crime_dta[, -"Crime"] %>% as.matrix, y = crime_dta[,      "Crime"] %>% as.matrix, nfolds = nrow(crime_dta), alpha = 0,      family = "gaussian") 

Measure: Mean-Squared Error 

    Lambda Index Measure    SE Nonzero
min  45.98    94   67889 12486      15
1se 295.54    74   79983 15210      15

In [159]:
options(warn=-1)

CV_err_list <- list()
counter <- 1

alpha_grid <- seq(0, 1, length=21)
for (alpha in alpha_grid) {
    mod <- cv.glmnet(x=crime_dta[, -"Crime"] %>% as.matrix,
                     y=crime_dta[, "Crime"] %>% as.matrix,
                     nfolds=nrow(crime_dta),
                     alpha=alpha, family="gaussian")
    CV_err_list[[counter]] <- data.table(alpha=alpha, lambda=mod$lambda.min, CV_err=mod$cvm[mod$index[1]])
    counter <- counter + 1
}
CV_err <- CV_err_list %>% rbindlist
CV_err

alpha,lambda,CV_err
<dbl>,<dbl>,<dbl>
0.00,45.976668,67888.93
0.05,41.699721,67551.41
0.10,36.435723,67458.89
0.15,29.257972,67438.34
0.20,26.430998,67574.58
0.25,23.206390,67771.06
0.30,19.338658,67940.14
0.35,18.192132,68079.01
0.40,15.918115,68198.12


### Elastic Net coincides with Lasso

In [167]:
CV_err[which.min(CV_err)]

alpha,lambda,CV_err
<dbl>,<dbl>,<dbl>
1,13.40244,65813.23


### Variable Selection in Final Lasso Model

`M`, `So`, `Ed`, `Po1`, `LF`, `M.F`, `NW`, `U1`, `U2`, `Ineq`, `Prob` are selected.

In [184]:
lasso_mod <- glmnet(x=crime_dta[, -"Crime"] %>% as.matrix,
                    y=crime_dta[, "Crime"] %>% as.matrix,
                    alpha=1, lambda=CV_err[which.min(CV_err), lambda], family="gaussian")

lasso_mod$beta

15 x 1 sparse Matrix of class "dgCMatrix"
                  s0
M         64.1560942
So        48.7815935
Ed        95.5282426
Po1      105.0198017
Po2        .        
LF        86.7265767
M.F       16.0218862
Pop        .        
NW         0.2318588
U1      -232.4988470
U2        42.8035013
Wealth     .        
Ineq      42.7106536
Prob   -3463.4094801
Time       .        